# Find proteins within the 8p loss event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "cis"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,brca_Database_ID,lscc_Database_ID,luad_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
ADAMDEC1,NP_055294.1|NP_001138744.1,NP_055294.1|NP_001138743.1,NP_055294.1|NP_001138743.1,NP_001138744,6.978531e-01,1.266649e-01,7.380721e-01,8.748058e-01,1.466949e-01,0.740499,-0.298616,-0.402267,-0.088317,0.084522,-1.167415,-0.155563
AGPAT5,NP_060831.2,NP_060831.2,NP_060831.2,NP_060831,2.144297e-03,2.549562e-03,9.045393e-04,9.115925e-03,3.360921e-04,0.010356,-0.849746,-0.293907,-0.274938,-0.578730,-0.940249,-0.327024
ANGPT2,NP_001138.1|NP_001112360.1|NP_001112359.1,NP_001112359.1|NP_001138.1,NP_001112359.1|NP_001138.1|NP_001112360.1,NP_001112359,5.032264e-01,NaN,7.922711e-01,2.733473e-01,3.227316e-01,0.314707,0.258756,NaN,0.050701,0.395267,-0.346771,-0.513799
ARHGEF10,NP_001295082.1|NP_055444.2|NP_001295081.1,NP_001295082.1,NP_001295082.1|NP_055444.2|NP_001295081.1,NP_001295081,4.226186e-04,5.620637e-01,2.327743e-02,2.733473e-01,1.497414e-03,0.032414,-0.728021,-0.085845,-0.189530,-0.245541,-0.599386,-0.277331
ASAH1,NP_004306.3|NP_808592.2,NP_004306.3|NP_808592.2|NP_001350672.1|NP_0011...,NP_808592.2|NP_004306.3|NP_001350672.1|NP_0011...,NP_004306,2.771828e-02,7.793495e-03,7.073915e-02,6.952236e-01,1.466949e-01,0.012499,-0.711515,-0.294897,-0.202186,-0.146084,-0.561108,-0.327443
ATP6V1B2,NP_001684.2,NP_001684.2,NP_001684.2,NP_001684,1.433931e-04,2.050610e-09,2.389824e-02,1.895422e-04,6.139620e-05,0.005998,-0.408102,-0.313560,-0.141487,-0.501203,-0.577990,-0.188970
BIN3,NP_061158.1,NP_061158.1|NP_001349975.1,NP_061158.1|NP_001349975.1,NP_061158,3.055856e-02,2.555672e-01,8.079039e-01,8.651408e-01,4.118205e-01,0.184275,-0.556700,-0.082998,-0.026597,-0.039881,-0.170859,-0.147306
BLK,NP_001706.2|NP_001317394.1,NaN,NP_001706.2|NP_001317394.1,NP_001317394,1.155079e-01,NaN,3.174666e-01,NaN,1.279759e-02,0.994758,0.441371,NaN,0.258200,NaN,-2.269184,-0.001206
BMP1,NP_006120.1|NP_001190.1,NP_006120.1|NP_001190.1,NP_006120.1|NP_001190.1,NP_006120,1.624506e-01,NaN,3.248692e-02,6.140621e-01,1.230674e-01,0.034368,-0.489983,NaN,0.398645,-0.295046,-0.740407,-0.424739


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,ADAMDEC1,NP_055294.1|NP_001138744.1,6.978531e-01,-0.298616
1,brca,AGPAT5,NP_060831.2,2.144297e-03,-0.849746
2,brca,ANGPT2,NP_001138.1|NP_001112360.1|NP_001112359.1,5.032264e-01,0.258756
3,brca,ARHGEF10,NP_001295082.1|NP_055444.2|NP_001295081.1,4.226186e-04,-0.728021
4,brca,ASAH1,NP_004306.3|NP_808592.2,2.771828e-02,-0.711515
5,brca,ATP6V1B2,NP_001684.2,1.433931e-04,-0.408102
6,brca,BIN3,NP_061158.1,3.055856e-02,-0.556700
7,brca,BLK,NP_001706.2|NP_001317394.1,1.155079e-01,0.441371
8,brca,BMP1,NP_006120.1|NP_001190.1,1.624506e-01,-0.489983
9,brca,BNIP3L,NP_004322.1|NP_001317420.1,1.985276e-02,-0.891026


## Select the proteins with a significant change, and take a detour to make some plots

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title=f"Chr {CHROMOSOME}{ARM} {TRANS_OR_CIS} effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [8]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,AGPAT5,NP_060831.2,2.144297e-03,-0.849746,-1
1,brca,ARHGEF10,NP_001295082.1|NP_055444.2|NP_001295081.1,4.226186e-04,-0.728021,-1
2,brca,ASAH1,NP_004306.3|NP_808592.2,2.771828e-02,-0.711515,-1
3,brca,ATP6V1B2,NP_001684.2,1.433931e-04,-0.408102,-1
4,brca,BIN3,NP_061158.1,3.055856e-02,-0.556700,-1
5,brca,BNIP3L,NP_004322.1|NP_001317420.1,1.985276e-02,-0.891026,-1
6,brca,CDCA2,NP_689775.2|NP_001304835.1|NP_001304836.1,4.767438e-03,-1.091624,-1
7,brca,CHMP7,NP_689485.1|NP_001304828.1,3.019346e-10,-0.837215,-1
8,brca,CNOT7,NP_001309022.1|NP_001309020.1|NP_001309023.1|N...,3.966029e-03,-0.380232,-1
9,brca,DOCK5,NP_079216.4,5.516873e-05,-0.758086,-1


In [9]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
AGPAT5,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
ATP6V1B2,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
CHMP7,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
ERI1,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
KIF13B,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
MSRA,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
NAT1,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
PPP2R2A,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
VPS37A,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6


In [10]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

6    10
5     6
4    10
3     7
2     8
1    17
Name: num_cancers, dtype: int64

## Save results

In [11]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")

## Info for most common proteins

- [AGPAT5](https://www.uniprot.org/uniprot/Q9NUQ2)  (colon, hnscc, lscc, luad, ovarian)     5
    - Converts 1-acyl-sn-glycerol-3-phosphate (lysophosphatidic acid or LPA) into 1,2-diacyl-sn-glycerol-3-phosphate (phosphatidic acid or PA) by incorporating an acyl moiety at the sn-2 position of the glycerol backbone (PubMed:21173190). Acts on LPA containing saturated or unsaturated fatty acids C15:0-C20:4 at the sn-1 position using C18:1-CoA as the acyl donor (PubMed:21173190). Also acts on lysophosphatidylethanolamine using oleoyl-CoA, but not arachidonoyl-CoA, and lysophosphatidylinositol using arachidonoyl-CoA, but not oleoyl-CoA (PubMed:21173190). Activity toward lysophosphatidylglycerol not detectable (PubMed:21173190).
- ATP6V1B2 	(colon, hnscc, lscc, luad, ovarian) 	5
- CCDC25 	(colon, hnscc, lscc, luad, ovarian) 	5
- CHMP7 	(colon, hnscc, lscc, luad, ovarian) 	5
- [ERI1](https://www.uniprot.org/uniprot/Q8IV48)    (colon, hnscc, lscc, luad, ovarian)     5
    - RNA exonuclease that binds to the 3'-end of histone mRNAs and degrades them, suggesting that it plays an essential role in histone mRNA decay after replication. A 2' and 3'-hydroxyl groups at the last nucleotide of the histone 3'-end is required for efficient degradation of RNA substrates. Also able to degrade the 3'-overhangs of short interfering RNAs (siRNAs) in vitro, suggesting a possible role as regulator of RNA interference (RNAi). Requires for binding the 5'-ACCCA-3' sequence present in stem-loop structure. Able to bind other mRNAs. Required for 5.8S rRNA 3'-end processing. Also binds to 5.8s ribosomal RNA. Binds with high affinity to the stem-loop structure of replication-dependent histone pre-mRNAs.
- [PPP2CB](https://www.uniprot.org/uniprot/P62714)  (colon, hnscc, lscc, luad, ovarian)     5
    - PP2A can modulate the activity of phosphorylase B kinase casein kinase 2, mitogen-stimulated S6 kinase, and MAP-2 kinase.
- [PPP2R2A](https://www.uniprot.org/uniprot/P63151)     (colon, hnscc, lscc, luad, ovarian)     5
    - The B regulatory subunit might modulate substrate selectivity and catalytic activity, and also might direct the localization of the catalytic enzyme to a particular subcellular compartment.
- [VPS37A](https://www.uniprot.org/uniprot/Q8NEZ2)  (colon, hnscc, lscc, luad, ovarian)     5
    - Component of the ESCRT-I complex, a regulator of vesicular trafficking process. Required for the sorting of endocytic ubiquitinated cargos into multivesicular bodies. May be involved in cell growth and differentiation.
- [XPO7](https://www.uniprot.org/uniprot/Q9UIA9)    (colon, hnscc, lscc, luad, ovarian)  5
    - Mediates the nuclear export of proteins (cargos) with broad substrate specificity. In the nucleus binds cooperatively to its cargo and to the GTPase Ran in its active GTP-bound form. Docking of this trimeric complex to the nuclear pore complex (NPC) is mediated through binding to nucleoporins. Upon transit of a nuclear export complex into the cytoplasm, disassembling of the complex and hydrolysis of Ran-GTP to Ran-GDP (induced by RANBP1 and RANGAP1, respectively) cause release of the cargo from the export receptor. XPO7 then return to the nuclear compartment and mediate another round of transport. The directionality of nuclear export is thought to be conferred by an asymmetric distribution of the GTP- and GDP-bound forms of Ran between the cytoplasm and nucleus.